In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
import sys
import os

sys.path.append('../tools')
import h5py
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch
from tqdm import tqdm
import sklearn

import torchvision.transforms as T
import pytorch_lightning as pl
import pytorch_lightning.loggers as pl_loggers
import pytorch_lightning.callbacks as pl_callbacks

import data_utility
import times
import segmentation
import preprocess
import autoencoder
import visualizer
import kaggle_data_utility
from kaggle_data_utility import KaggleDataset

In [3]:
data_dir = "../../../user_data/competition_data/clips"
log_folder_root = '../../../user_data/logs/'
ckpt_folder_root = '../../../user_data/checkpoints/'

targets = [
    'Dog_1',
    'Dog_2',
    'Dog_3',
    'Dog_4',
]

# targets = [
#     'Patient_1',
#     'Patient_2',
#     'Patient_3',
#     'Patient_4',
#     'Patient_5',
#     'Patient_6',
#     'Patient_7',
#     'Patient_8'
# ]

In [4]:
random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)
determine_generator = torch.Generator()
determine_generator.manual_seed(random_seed)

if torch.cuda.is_available():
    torch.cuda.manual_seed(random_seed)
    # True ensures the algorithm selected by CUFA is deterministic
    torch.backends.cudnn.deterministic = True
    # torch.set_deterministic(True)
    # False ensures CUDA select the same algorithm each time the application is run
    torch.backends.cudnn.benchmark = False

In [5]:
ictal_data_list = [kaggle_data_utility.parse_input_data(data_dir, targets[i], 'ictal', None) for i in
                   range(len(targets))]
interictal_data_list = [kaggle_data_utility.parse_input_data(data_dir, targets[i], 'interictal', None) for i in
                        range(len(targets))]
test_data_list = [kaggle_data_utility.parse_input_data(data_dir, targets[i], 'test', None) for i in range(len(targets))]

Loading data
(0s)
X (178, 16, 400) y (178,) latencies (178,)
Loading data
(0s)
X (172, 16, 400) y (172,) latencies (172,)
Loading data
(0s)
X (480, 16, 400) y (480,) latencies (480,)
Loading data
(0s)
X (257, 16, 400) y (257,) latencies (257,)
Loading data
(0s)
X (418, 16, 400) y (418,)
Loading data
(0s)
X (1148, 16, 400) y (1148,)
Loading data
(2s)
X (4760, 16, 400) y (4760,)
Loading data
(1s)
X (2790, 16, 400) y (2790,)
Loading data
(1s)
X (3181, 16, 400)
Loading data
(1s)
X (2997, 16, 400)
Loading data
(2s)
X (4450, 16, 400)
Loading data
(1s)
X (3013, 16, 400)


In [6]:
ictal_data_X = np.empty((0, 16, 400))
interictal_data_X = np.empty((0, 16, 400))
test_data_X = np.empty((0, 16, 400))
for data in ictal_data_list:
    ictal_data_X = np.vstack((ictal_data_X, data['X']))
for data in interictal_data_list:
    interictal_data_X = np.vstack((interictal_data_X, data['X']))
for data in test_data_list:
    test_data_X = np.vstack((test_data_X, data['X']))

In [7]:
unlabeled_dataset = KaggleDataset(ictal_data_X, interictal_data_X, test_data_X, labeled=False, transform=True,astensor = False)
train_set_size = int(unlabeled_dataset.length * 0.8)
valid_set_size = unlabeled_dataset.length - train_set_size
train_set, test_set = torch.utils.data.random_split(unlabeled_dataset, [train_set_size, valid_set_size],generator=determine_generator)

In [8]:
unlabeled_dataset.length

23844

In [9]:
# import torch
# import torchvision
# from torch import nn
#
# from lightly.data import LightlyDataset, SwaVCollateFunction
# from lightly.loss import SwaVLoss
# from lightly.loss.memory_bank import MemoryBankModule
# from lightly.models.modules import SwaVProjectionHead, SwaVPrototypes
#
#
# class SwaV(nn.Module):
#     def __init__(self, backbone):
#         super().__init__()
#         self.backbone = backbone
#         self.projection_head = SwaVProjectionHead(512, 512, 128)
#         self.prototypes = SwaVPrototypes(128, 512, 5)
#
#         self.start_queue_at_epoch = 30
#         self.queues = nn.ModuleList([MemoryBankModule(size=512) for _ in range(2)])
#
#     def forward(self, high_resolution, low_resolution, epoch):
#         self.prototypes.normalize()
#
#         high_resolution_features = [self._subforward(x) for x in high_resolution]
#         low_resolution_features = [self._subforward(x) for x in low_resolution]
#
#         high_resolution_prototypes = [
#             self.prototypes(x, epoch) for x in high_resolution_features
#         ]
#         low_resolution_prototypes = [
#             self.prototypes(x, epoch) for x in low_resolution_features
#         ]
#         queue_prototypes = self._get_queue_prototypes(high_resolution_features, epoch)
#
#         return high_resolution_prototypes, low_resolution_prototypes, queue_prototypes
#
#     def _subforward(self, input):
#         features = self.backbone(input).flatten(start_dim=1)
#         features = self.projection_head(features)
#         features = nn.functional.normalize(features, dim=1, p=2)
#         return features
#
#     @torch.no_grad()
#     def _get_queue_prototypes(self, high_resolution_features, epoch):
#         if len(high_resolution_features) != len(self.queues):
#             raise ValueError(
#                 f"The number of queues ({len(self.queues)}) should be equal to the number of high "
#                 f"resolution inputs ({len(high_resolution_features)}). Set `n_queues` accordingly."
#             )
#
#         # Get the queue features
#         queue_features = []
#         for i in range(len(self.queues)):
#             _, features = self.queues[i](high_resolution_features[i], update=True)
#             # Queue features are in (num_ftrs X queue_length) shape, while the high res
#             # features are in (batch_size X num_ftrs). Swap the axes for interoperability.
#             features = torch.permute(features, (1, 0))
#             queue_features.append(features)
#
#         # If loss calculation with queue prototypes starts at a later epoch,
#         # just queue the features and return None instead of queue prototypes.
#         if self.start_queue_at_epoch > 0 and epoch < self.start_queue_at_epoch:
#             return None
#
#         # Assign prototypes
#         queue_prototypes = [self.prototypes(x, epoch) for x in queue_features]
#         return queue_prototypes
#
#
# resnet = torchvision.models.resnet34()
#
# backbone = nn.Sequential(*list(resnet.children())[:-1])
# model = SwaV(backbone)
#
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

# collate_fn = SwaVCollateFunction(gaussian_blur = 0, hf_prob = 0,vf_prob = 0,rr_prob=0,cj_prob=0,random_gray_scale=0, normalize={'mean':[0, 0, 0], 'std':[1, 1, 1]})
#
# dataloader = torch.utils.data.DataLoader(
#     unlabeled_dataset,
#     batch_size=128,
#     collate_fn=collate_fn,
#     shuffle=True,
#     drop_last=True,
# )
#
# criterion = SwaVLoss(sinkhorn_epsilon = 0.05)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#
# # torch.save({
# #             'epoch': -1,
# #             'model_state_dict': model.state_dict(),
# #             'optimizer_state_dict': optimizer.state_dict(),
# #             'loss': -1,
# #             }, 'ckpt/checkpoint_round_2'+str(-1)+'.pth')
#
# print("Starting Training")
# for epoch in range(100):
#     total_loss = 0
#     i = 0
#     for batch, _, _ in tqdm(dataloader):
#         batch = [x.to(device) for x in batch]
#         high_resolution, low_resolution = batch[:2], batch[2:]
#         high_resolution, low_resolution, queue = model(
#             high_resolution, low_resolution, epoch
#         )
#         loss = criterion(high_resolution, low_resolution, queue)
#         total_loss += loss.detach()
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()
#
#     avg_loss = total_loss / len(dataloader)
#     torch.save({
#             'epoch': epoch,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'loss': avg_loss,
#             }, 'kaggle_human_ckpt/checkpoint'+str(epoch)+'.pth')
#
#     print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")

In [10]:
from models.SwaV import SwaV
from lightly.data import SwaVCollateFunction

model = SwaV()

# model.load_from_checkpoint()

collate_fn = SwaVCollateFunction(gaussian_blur = 0, hf_prob = 0,vf_prob = 0,rr_prob=0,cj_prob=0,random_gray_scale=0, normalize={'mean':[0, 0, 0], 'std':[1, 1, 1]})

dataloader = torch.utils.data.DataLoader(
    unlabeled_dataset,
    batch_size=128,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True,
)
accelerator = "gpu" if torch.cuda.is_available() else "cpu"

checkpoint_callback = pl_callbacks.ModelCheckpoint(monitor='swav_loss',filename='kaggle_dog_swav-{epoch:02d}-{swav_loss:.5f}', save_last=True, save_top_k=-1, dirpath=ckpt_folder_root + 'kaggle_dog_swav')
csv_logger = pl_loggers.CSVLogger(log_folder_root, name="kaggle_dog_swav")

trainer = pl.Trainer(logger=csv_logger, max_epochs=120, callbacks=[checkpoint_callback],accelerator='gpu', devices=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model=model, train_dataloaders=dataloader)

C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:616: UserWarning: Checkpoint directory C:\Users\Patrick Xu\Desktop\RNS_Annotation-Pipeline\user_data\checkpoints\kaggle_dog_swav exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type               | Params
-------------------------------------------------------
0 | backbone        | Sequential         | 21.3 M
1 | projection_head | SwaVProjectionHead | 328 K 
2 | prototypes      | SwaVPrototypes     | 66.0 K
3 | queues          | ModuleList         | 0     
4 | criterion       | SwaVLoss           | 0     
-------------------------------------------------------
21.7 M    Trainable params
0         Non-trainable params
21.7 M    Total params
86.718    Total estimated model params size (MB)
C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python31

Training: 0it [00:00, ?it/s]

In [12]:
import torch


def sigmoid_focal_loss(
    inputs: torch.Tensor,
    targets: torch.Tensor,
    alpha: float = 0.25,
    gamma: float = 2,
    reduction: str = "none",
) -> torch.Tensor:
    """
    Loss used in RetinaNet for dense detection: https://arxiv.org/abs/1708.02002.

    Args:
        inputs (Tensor): A float tensor of arbitrary shape.
                The predictions for each example.
        targets (Tensor): A float tensor with the same shape as inputs. Stores the binary
                classification label for each element in inputs
                (0 for the negative class and 1 for the positive class).
        alpha (float): Weighting factor in range (0,1) to balance
                positive vs negative examples or -1 for ignore. Default: ``0.25``.
        gamma (float): Exponent of the modulating factor (1 - p_t) to
                balance easy vs hard examples. Default: ``2``.
        reduction (string): ``'none'`` | ``'mean'`` | ``'sum'``
                ``'none'``: No reduction will be applied to the output.
                ``'mean'``: The output will be averaged.
                ``'sum'``: The output will be summed. Default: ``'none'``.
    Returns:
        Loss tensor with the reduction option applied.
    """
    # Original implementation from https://github.com/facebookresearch/fvcore/blob/master/fvcore/nn/focal_loss.py

    p = torch.sigmoid(inputs)

    ce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction="none")
    p_t = p * targets + (1 - p) * (1 - targets)
    loss = ce_loss * ((1 - p_t) ** gamma)

    if alpha >= 0:
        alpha_t = alpha * targets + (1 - alpha) * (1 - targets)
        loss = alpha_t * loss

    # Check reduction option and return loss accordingly
    if reduction == "none":
        pass
    elif reduction == "mean":
        loss = loss.mean()
    elif reduction == "sum":
        loss = loss.sum()
    else:
        raise ValueError(
            f"Invalid Value for arg 'reduction': '{reduction} \n Supported reduction modes: 'none', 'mean', 'sum'"
        )
    return loss

In [13]:
# unlabeled_dataset = KaggleDataset(ictal_data_X, interictal_data_X, test_data_X, labeled=False, transform=True)
# train_set_size = int(labeled_dataset.length * 0.8)
# valid_set_size = labeled_dataset.length - train_set_size
# train_set, test_set = torch.utils.data.random_split(labeled_dataset, [train_set_size, valid_set_size])

In [14]:
import torch.nn.functional as F
class SupervisedDownstream(pl.LightningModule):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 8)
        self.fc4 = nn.Linear(8, 2)
        self.softmax = nn.Softmax(dim=1)
        self.alpha = 0.5
        self.gamma = 8


    def training_step(self, batch, batch_idx):
        x, y = batch
        # self.backbone.eval()
        x = self.backbone(x)
        # with torch.no_grad():
        x = x.view(-1,512)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        pred = self.fc4(x)
        pred = self.softmax(pred)
        label = F.one_hot(y).squeeze()
        loss = sigmoid_focal_loss(pred.float(),label.float(), alpha = self.alpha, gamma = self.gamma,reduction = 'mean')
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = self.backbone(x)
        x = x.view(-1,512)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        pred = self.fc4(x)
        pred = self.softmax(pred)
        label = F.one_hot(y).squeeze()
        loss = sigmoid_focal_loss(pred.float(),label.float(), alpha = self.alpha, gamma = self.gamma,reduction = 'mean')
        out = torch.argmax(pred, dim=1)
        out = out.detach().cpu().numpy()
        target = y.squeeze().detach().cpu().numpy()
        precision,recall,fscore,support = sklearn.metrics.precision_recall_fscore_support(out, target)
        acc=sklearn.metrics.accuracy_score(out, target)
        # Logging to TensorBoard (if installed) by default
        self.log("val_loss", loss)
        self.log("val_acc", acc)
        self.log("val_precision", precision[1])
        self.log("val_recall", recall[1])
        return pred, label

    def predict_step(self, batch, batch_idx):
        x, y = batch
        emb = self.backbone(x)
        emb = emb.view(-1,512)
        x = F.relu(self.fc1(emb))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        pred = self.fc4(x)
        # Logging to TensorBoard (if installed) by default
        return pred, y, emb

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [15]:
ckpt = torch.load("ckpt/checkpoint99.pth")
resnet = torchvision.models.resnet34()
backbone = nn.Sequential(*list(resnet.children())[:-1])
swav = SwaV(backbone)
swav.load_state_dict(ckpt['model_state_dict'])
model = SupervisedDownstream(swav.backbone)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

checkpoint_callback = pl_callbacks.ModelCheckpoint(monitor='val_loss',filename='swav_pretrained_unfreeze-{epoch:02d}-{val_loss:.5f}',dirpath='checkpoints')
csv_logger = pl_loggers.CSVLogger("logs", name="logger")

trainer = pl.Trainer( logger=csv_logger, max_epochs=80, callbacks=[checkpoint_callback],accelerator='gpu', devices=1)


FileNotFoundError: [Errno 2] No such file or directory: 'ckpt/checkpoint99.pth'

In [ ]:

labeled_dataset = KaggleDataset(ictal_data_X, interictal_data_X, test_data_X, labeled=True, transform=True)
train_set_size = int(labeled_dataset.length * 0.8)
valid_set_size = labeled_dataset.length - train_set_size
train_set, test_set = torch.utils.data.random_split(labeled_dataset, [train_set_size, valid_set_size],generator=torch.Generator().manual_seed(42))

def collate_fn(batch):
    info = list(zip(*batch))
    data = info[0]
    label = info[1]
    return torch.stack(data), torch.stack(label)

train_dataloader = torch.utils.data.DataLoader(
    train_set,
    batch_size=128,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True,
)
val_dataloader = torch.utils.data.DataLoader(
    test_set,
    batch_size=128,
    collate_fn=collate_fn,
    shuffle=False,
    drop_last=True,
)
trainer.fit(model, train_dataloader, val_dataloader)

In [ ]:
labeled_dataset = KaggleDataset(ictal_data_X, interictal_data_X, test_data_X, labeled=True, transform=False)
train_set_size = int(labeled_dataset.length * 0.8)
valid_set_size = labeled_dataset.length - train_set_size
train_set, test_set = torch.utils.data.random_split(labeled_dataset, [train_set_size, valid_set_size],generator=torch.Generator().manual_seed(42))
train_dataloader = torch.utils.data.DataLoader(
    train_set,
    batch_size=128,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True,
)
val_dataloader = torch.utils.data.DataLoader(
    test_set,
    batch_size=128,
    collate_fn=collate_fn,
    shuffle=False,
    drop_last=True,
)
predictions = trainer.predict(model,val_dataloader,ckpt_path='checkpoints/swav_pretrained-epoch=26-val_loss=0.00029_alpha_0.9_gamma_8_sftm.ckpt')

In [ ]:
train_set

In [ ]:
output_list = []
target_list = []
m = nn.Softmax(dim=1)
for pred, y,_ in predictions:
    out = m(pred)
    print(out)
    output_list.append(out)
    target_list.append(y)

In [ ]:
output = torch.vstack(output_list)
target = torch.vstack(target_list)
inds = np.where(target==1)[0]

fpr, tpr, thresholds = sklearn.metrics.roc_curve(target,output[:,1],pos_label=1)
sklearn.metrics.auc(fpr, tpr)

In [ ]:
output[:,1][inds]

In [ ]:
from sklearn.metrics import RocCurveDisplay
RocCurveDisplay.from_predictions(
    target,
    output[:,1],
    color="darkorange",
)
plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves:\nVirginica vs (Setosa & Versicolor)")
plt.legend()
plt.show()

In [ ]:
output[:,1]

In [ ]:

output = torch.argmax(output, dim=1)
output = output.detach().cpu().numpy()
target= target.squeeze().detach().cpu().numpy()

In [ ]:
import sklearn

clf_report = sklearn.metrics.classification_report(output, target, digits=6)

print(f"Classification Report : \n{clf_report}")

In [ ]:
for batch, label in tqdm(val_dataloader):
        batch = batch.to(device)
        label = label.to(device)
        label = F.one_hot(label).squeeze()
        outputs = model(batch)
        print(batch)
        loss = sigmoid_focal_loss(pred.float(),label.float(), alpha = 0.5, gamma = 8,reduction = 'mean')
        print(loss)
        break

In [ ]:
# import copy
# import torch
# import torchvision
# from torch import nn
#
# from lightly.data import DINOCollateFunction, LightlyDataset
# from lightly.loss import DINOLoss
# from lightly.models.modules import DINOProjectionHead
# from lightly.models.utils import deactivate_requires_grad, update_momentum
# from lightly.utils.scheduler import cosine_schedule
#
#
# class DINO(torch.nn.Module):
#     def __init__(self, backbone, input_dim):
#         super().__init__()
#         self.student_backbone = backbone
#         self.student_head = DINOProjectionHead(
#             input_dim, 512, 64, 2048, freeze_last_layer=1
#         )
#         self.teacher_backbone = copy.deepcopy(backbone)
#         self.teacher_head = DINOProjectionHead(input_dim, 512, 64, 2048)
#         deactivate_requires_grad(self.teacher_backbone)
#         deactivate_requires_grad(self.teacher_head)
#
#     def forward(self, x):
#         y = self.student_backbone(x).flatten(start_dim=1)
#         z = self.student_head(y)
#         return z
#
#     def forward_teacher(self, x):
#         y = self.teacher_backbone(x).flatten(start_dim=1)
#         z = self.teacher_head(y)
#         return z
#
#
# resnet = torchvision.models.resnet18()
# backbone = nn.Sequential(*list(resnet.children())[:-1])
# input_dim = 512
# # instead of a resnet you can also use a vision transformer backbone as in the
# # original paper (you might have to reduce the batch size in this case):
# # backbone = torch.hub.load('facebookresearch/dino:main', 'dino_vits16', pretrained=False)
# # input_dim = backbone.embed_dim
#
# model = DINO(backbone, input_dim)
#
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)
#
# # # we ignore object detection annotations by setting target_transform to return 0
# # pascal_voc = torchvision.datasets.VOCDetection(
# #     "datasets/pascal_voc", download=True, target_transform=lambda t: 0
# # )
# # dataset = LightlyDataset.from_torch_dataset(pascal_voc)
# # # or create a dataset from a folder containing images or videos:
# # # dataset = LightlyDataset("path/to/folder")
#
# collate_fn = DINOCollateFunction(solarization_prob = 0, hf_prob = 0,vf_prob = 0,rr_prob=0,cj_prob=0,random_gray_scale=0)
#
# dataloader = torch.utils.data.DataLoader(
#     train_set,
#     batch_size=64,
#     collate_fn=collate_fn,
#     shuffle=True,
#     drop_last=True,
#     num_workers=1,
# )
#
# criterion = DINOLoss(
#     output_dim=2048,
#     warmup_teacher_temp_epochs=5,
# )
# # move loss to correct device because it also contains parameters
# criterion = criterion.to(device)
#
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#
# epochs = 10
#
# print("Starting Training")
# for epoch in range(epochs):
#     total_loss = 0
#     momentum_val = cosine_schedule(epoch, epochs, 0.996, 1)
#     for views, _, _ in tqdm(dataloader):
#         update_momentum(model.student_backbone, model.teacher_backbone, m=momentum_val)
#         update_momentum(model.student_head, model.teacher_head, m=momentum_val)
#         views = [view.to(device) for view in views]
#         global_views = views[:2]
#         teacher_out = [model.forward_teacher(view) for view in global_views]
#         student_out = [model.forward(view) for view in views]
#         loss = criterion(teacher_out, student_out, epoch=epoch)
#         total_loss += loss.detach()
#         loss.backward()
#         # We only cancel gradients of student head.
#         model.student_head.cancel_last_layer_gradients(current_epoch=epoch)
#         optimizer.step()
#         optimizer.zero_grad()
#
#     avg_loss = total_loss / len(dataloader)
#     print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")

In [ ]:
augmentation = T.Compose([
    T.ToPILImage(),
    T.Resize((256, 512), interpolation=T.InterpolationMode.NEAREST),
    T.RandomApply([T.ColorJitter()], p=0.5),
    T.RandomApply([T.GaussianBlur(kernel_size=(3, 3))], p=0.5),
    T.RandomInvert(p=0.2),
    T.RandomPosterize(4, p=0.2),
])

data = ictal_data_X[0]

channel_index = np.arange(data.shape[0])
np.random.shuffle(channel_index)
data = data[channel_index]
data = torch.from_numpy(data).clone()
data = data.repeat(3, 1, 1)
data = augmentation(data)
data

In [ ]:
channel_index

In [ ]:
data[channel_index]

In [ ]:
data

In [ ]:
#
# print("Starting Training")
# for epoch in range(50):
#     total_loss = 0
#     i = 0
#     for batch, label in tqdm(dataloader):
#         batch = batch.to(device)
#         # print(type(batch))
#         label = label.to(device)
#         label = F.one_hot(label).squeeze()
#         outputs = model(batch)
#         loss = sigmoid_focal_loss(outputs.float(),label.float(), alpha = 0.25, gamma = 7,reduction = 'mean')
#         total_loss += loss.detach()
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()
#
#     avg_loss = total_loss / len(dataloader)
#     torch.save({
#             'epoch': epoch,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'loss': avg_loss,
#             }, 'ckpt/checkpoint'+str(epoch)+'.pth')
#
#     print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")